# Reinforcement Q-learning with RNN as function approximation

- portfolio grid of size 10 (0 to 1)
- 2 hidden neural layers with the first one being the recurrent layer (also has the weights of the previous 3 states as input)
- Improved by Dropout 
- Transaction costs 

In [1]:
# initialization
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from random import randint
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# import real data
mdata = pd.read_csv('data.csv') #Three stocks (R,X_s,X_b) Without predictors
mdata = np.array(mdata[['r','xs','xb']])

In [3]:
#hyperparameters
series_length = 15                                           
batch_size = 1                                              
truncated_backprop_length = series_length//batch_size       # How many previous stock returns to take as input for the model
state_size = 4                                              # Number of nodes in the first hidden layer
num_classes = 10                                            # Number of classes to predict (10 actions possible so 10 classes)
echo_step = 3                                               # How many previous states the neural network takes as input
num_stocks = 3                                              # Amount of stocks into consideration
gamma = 1                                                   # Discount factor of future Q-values
epsilon = 0.1                                               # For the randomization of actions
n = mdata.size/4-4                                          # Length of the total data
periods = 60                                                # How many periods in the future to predict
epochs = 1                                                  # Amount of iterations to train the Neural Network
TC = 0                                                      # Percentage of transaction costs
dropout_prob = (1 - 0.5)                                    # Percentage of neuron nodes to keep in the Network using dropout

# Initialization of the Tensorflow placeholders and the network

In [4]:
#Initialize Recurrent Neural Network and set-up the placeholders
tf.reset_default_graph()
NN_input = tf.placeholder(tf.float32, [num_stocks, batch_size, truncated_backprop_length])
init_state = tf.placeholder(tf.float32, [num_stocks, batch_size, state_size])
Q_Next = tf.placeholder(tf.float32, [truncated_backprop_length, batch_size, num_classes])

# Weights and biases
W = tf.Variable(np.random.rand(num_stocks, state_size+1, state_size), dtype=tf.float32)
b = tf.Variable(np.zeros((num_stocks,1,state_size)), dtype=tf.float32)
W2 = tf.Variable(np.random.rand(num_stocks, state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((num_stocks, 1,num_classes)), dtype=tf.float32)

inputs_series = tf.unstack(NN_input, axis=2)
labels_series = tf.unstack(Q_Next, axis=0)

#Forward pass
current_state = init_state
states_series = []

for current_input in inputs_series:
    current_input = tf.reshape(current_input, [num_stocks,batch_size, 1])
    input_and_state_concatenated = tf.concat([current_input, current_state],axis=2)  # Increasing number of columns
    next_state = tf.tanh(tf.matmul(input_and_state_concatenated,W) + b)  # Broadcasted addition
    dropout_state = tf.nn.dropout(next_state,dropout_prob)     # DROPOUT
    states_series.append(dropout_state)
    current_state = next_state

#calculate loss
Q_FA = [tf.matmul(state, W2) + b2 for state in states_series]
dropout_Q = tf.nn.dropout(Q_FA,dropout_prob)    # DROPOUT
Q_FA = tf.reduce_sum(dropout_Q,axis=1)
A_Max = tf.argmax(Q_FA[-1],1) # only use the latest Q of the RNN for the determination of the optimal weights
Q_series = tf.unstack(Q_FA, axis=0)

# Calculate loss for the NN from the Q values
losses = [ abs(logits - labels) for logits, labels in zip(Q_series,labels_series)]
total_loss = tf.reduce_mean(losses)
# loss = tf.reduce_sum(tf.square(Q_Next - Q_FA))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(total_loss)

#Define Action Matrix (Now discrete case) 
A = np.linspace(0,1,10) # portfolio weights of stocks (1-weight) is the weight in the bonds

# Training of the RNN function approximation

In [ ]:
# initialization for insight series
TWlist = []
Index = []
MWeights = []

#
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    for i in range(341,int(n-periods-1)):
        OptimalWeights = np.zeros(periods-1)
        currentK = 0;
        print(i)
        sess.run(tf.global_variables_initializer()) # initialize the Neural Network again
        
        while currentK < periods - 1:
            #Initilization
            NN_data = mdata[0:i+currentK]  # rolling window
            #initialize an empty hidden state
            _current_state = np.zeros((num_stocks, batch_size, state_size))
            # NN_data = mdata[0:i+currentK]   #Expanding window
            rAll = 0
            currentEpoch = 0
            
            while currentEpoch < epochs:
                a_old = 0
                #Training of the Q-Network for the data available (with Neural Nets) 
                for j in range(0,len(NN_data)-truncated_backprop_length-1):
                    s = NN_data[j:j+truncated_backprop_length,0:3].reshape(num_stocks,batch_size,truncated_backprop_length)
                    #Choose an action by greedily (with e chance of random action) from the Q-network
                    a_int,allQ = sess.run([A_Max,Q_FA],feed_dict={NN_input:s, init_state:_current_state})
                    a = A[a_int-1]  # -1 because the output neurons are labeled 1 till 101 and it will be an index
                    if np.random.rand(1) < epsilon:
                        a = random.choice(A)

                    #Get new state and reward from environment
                    s1 = NN_data[j+truncated_backprop_length:j+truncated_backprop_length+1,1:4]
#                   s1 = mdata[i+currentK+j+truncated_backprop_length+1,1:4]
                    r = (a*s1[0][0] + (1-a)*s1[0][1] - TC*abs(a-a_old)) #reward: this is now the wealth gained from this step, but could be other rewards like utility
                    
                    ## idea: ADD also exp(s1) to the transaction costs
                    a_old = a
                    s1 = NN_data[j+1:j+truncated_backprop_length+1,0:3].reshape(num_stocks,batch_size,truncated_backprop_length)
                    Q = sess.run(Q_FA,feed_dict={NN_input:s1, init_state:_current_state})
                    print(sess.run(Q_FA,feed_dict={NN_input:s1, init_state:_current_state}))
                    #Obtain maxQ' and set our target value for chosen action.
                    Q1 = np.max(Q[-1])
                    targetQ = allQ
                    targetQ[-1,0,a_int] = r + gamma*Q1
            
            
                    ### idea: Print the losses 
            
            
                    #Train the neural network using target and predicted Q values
                    s = s1
                    _,_current_state,W1 = sess.run([updateModel,current_state,NN_input],feed_dict={NN_input:s,Q_Next:targetQ, init_state:_current_state})
                currentEpoch += 1
        
            # After training now calculate the optimal weights for the K=60 periods to come
            s1 = NN_data[j:j+truncated_backprop_length,0:3].reshape(num_stocks,batch_size,truncated_backprop_length)
            a_int,allQ = sess.run([A_Max,Q_FA],feed_dict={NN_input:s, init_state:_current_state})
            aOpt = A[a_int-1]
            OptimalWeights[currentK] = aOpt
            currentK += 1
        
        # For insight purposes
        MWeights.append(np.mean(OptimalWeights))
#         x = mdata[i+currentK+j+truncated_backprop_length+1,1:4]
        x = mdata[j+ truncated_backprop_length - currentK + 1:j+truncated_backprop_length+1,1:4]
        TWlist.append(np.exp(sum(OptimalWeights*x[:,0] + (1-OptimalWeights)*x[:,1])))
        Index.append(i)
        
        print('Writing away results')
        df = pd.DataFrame({'index date':Index,'TW':TWlist, 'Mean Weights Xs':MWeights})
        df.to_excel('Results_ExpRNN_g10_Other_new2.xlsx', sheet_name='sheet1', index=False)
        print(TWlist[-1])
        print(OptimalWeights)
        
    # close session
plt.plot(MWeights)
plt.plot(TWlist)

341
[[[  3.71215935e-03   7.07663421e-04   2.25606817e-03   0.00000000e+00
     6.58138830e-04   6.46987348e-04   1.03093730e-03   2.35820794e-03
     1.70179037e-03   1.72522920e-03]]

 [[  0.00000000e+00   3.21454555e-02   0.00000000e+00   0.00000000e+00
    -4.66342159e-02   3.49890552e-02  -5.25257587e-02   1.48029864e-01
    -4.73028757e-02   2.96943821e-02]]

 [[ -3.95396873e-02  -1.76558439e-02   3.13250422e-02   2.79518273e-02
     6.25328422e-02  -1.20350435e-01   2.66496036e-02   2.01360267e-02
    -1.23054609e-01   0.00000000e+00]]

 [[ -2.86444463e-03   5.69440648e-02   5.71506992e-02   2.21158634e-03
     0.00000000e+00  -2.95727253e-01  -2.30752259e-01   4.89620753e-02
     1.36360768e-02   1.01032495e-01]]

 [[  1.78958163e-01   1.48865014e-01   5.47345638e-01   0.00000000e+00
     1.63248867e-01  -4.87638831e-01   1.35377675e-01   2.35952705e-01
     5.99159226e-02   4.18455601e-02]]

 [[  4.13313866e-01   8.71979669e-02  -1.80587769e+00   5.38590848e-01
    -1.27242279

     2.73185253e+00  -5.86666726e-02]]]
[[[  1.63435459e+00   2.19384503e+00   6.53298473e+00   8.74836731e+00
     4.57174444e+00   1.18114319e+01   1.19501209e+00   6.40385389e+00
     2.02393055e+00   3.28295231e-01]]

 [[  3.94762254e+00   0.00000000e+00  -4.00000066e-02   0.00000000e+00
     8.80373859e+00   1.05484028e+01   1.69838130e+00   8.52081394e+00
    -9.33333412e-02   3.17191529e+00]]

 [[  1.89357281e+00   4.05695963e+00   2.81966853e+00   2.72150254e+00
     5.70492411e+00   0.00000000e+00   1.93918157e+00   7.37085819e+00
     0.00000000e+00   5.12169421e-01]]

 [[  0.00000000e+00   4.57159185e+00   1.38877468e+01   7.81040955e+00
     8.43410587e+00   1.58624516e+01   1.11438618e+01   0.00000000e+00
     4.89492702e+00   5.12145996e-01]]

 [[  5.20613003e+00   5.57296896e+00   3.11585164e+00   1.60559297e+00
     4.18990105e-03   0.00000000e+00   0.00000000e+00   9.98902798e+00
     6.81013703e-01   3.29937100e+00]]

 [[  0.00000000e+00   2.12499046e+00   1.89032173e

[[[ -0.20800002   2.958251    -0.09600002   0.          -0.14133334
     3.39785862   6.47021103   5.02201271   0.25313708   1.09251022]]

 [[  6.66415405   5.30996609   8.11987782   2.8772676    2.80143166
     8.90793705   7.62869978   7.59938717   4.1791544    0.        ]]

 [[  0.           4.18486691   3.33847547   2.66255283   0.           4.88746881
     1.11377835   4.74579048   3.79924536   4.00792456]]

 [[  1.78248692   1.60953403   0.           5.29124641   4.31279898
     5.97182274   1.78375053   0.           0.47842312   2.47922635]]

 [[  3.07411408   2.38032961   6.67127371   2.58863354   6.82191706
    15.61953735   0.           9.86814308   1.61918712   6.89139175]]

 [[  0.26196104   2.90820885   4.69769573   0.69511396   5.67069244
     1.82518673   2.61950731   6.56127357  -0.25536367   0.02153155]]

 [[  2.4747293    7.26197815   1.74931526   5.73649597   6.96723557
    10.67234612   4.6612072    1.9030478    3.42456269  11.11053181]]

 [[  0.           8.9983806

[[[  0.           0.79075873   6.62374783   6.4787178    4.53858757
     3.83744335   4.44239378   3.87278056   3.06476355   0.18437141]]

 [[  4.5820241    1.64196229   2.18537092   1.4047637    3.51507998
     9.51016617   6.68264341   4.96399832   1.34787178   4.24128866]]

 [[  6.47484684   1.11241603   2.90569258   8.55409908   7.33839989
     4.99446344   1.98046708   2.16245675   4.28303909   0.        ]]

 [[  0.           2.8886528    3.35954666   0.           7.2197814
     5.71452045   1.70045614   0.           2.64771628   1.7944833 ]]

 [[  2.61279249   2.63346004   2.97894597   4.38392401   4.45215225
     0.61646312   6.25429296   1.65967786   0.17962362   4.70221376]]

 [[  0.79199374   1.01197898   0.           1.68704915   8.43211651
     2.60889935   2.14971709   0.91558552   3.7262907    3.63147449]]

 [[  0.19173186   3.32754755   3.14177942   2.56583238   4.68560266
     3.49156618   2.17130733   0.9077915    0.           3.48605728]]

 [[  3.81949568   1.85402811

    1.63027775  5.35018826 -1.10862446  2.67787075]]]
[[[  1.12202585e+00   2.21122360e+00   2.30566645e+00   8.09190571e-02
     5.16034985e+00   2.41795540e+00   0.00000000e+00   1.46750653e+00
    -9.61071849e-01   1.35733771e+00]]

 [[  2.07666922e+00   1.04486656e+00   2.66393805e+00   1.47109151e+00
     3.30344701e+00   0.00000000e+00   1.91465878e+00   2.19673443e+00
     1.40642643e+00   2.00367236e+00]]

 [[  1.84205675e+00   2.78536129e+00   4.24930382e+00   6.82010651e+00
     2.28863502e+00   4.59065485e+00   3.52520013e+00   0.00000000e+00
     5.82306981e-01   2.86420822e+00]]

 [[  3.28635597e+00  -1.39640081e+00   2.39356446e+00   2.85401672e-01
     0.00000000e+00   1.29403043e+00   0.00000000e+00   0.00000000e+00
     2.40672612e+00   5.10469079e-03]]

 [[  1.30207229e+00   1.26498973e+00   2.32424974e+00  -1.83256865e-01
     4.14271736e+00   2.16845202e+00   2.12812042e+00   1.45994401e+00
    -4.33589101e-01   5.03797531e-01]]

 [[  3.17668223e+00   1.87657976e+00

...

    -2.96425700e-01   0.00000000e+00]]]
[[[-0.06001428 -0.05773865 -0.28957856 -0.59615421 -0.25312006 -0.33942831
   -0.16254631 -0.08749041 -0.23612233 -0.14937288]]

 [[-0.13546795 -0.09314948 -0.09405249 -0.24905945 -0.1234408   0.          0.
   -0.01336062  0.         -0.1864368 ]]

 [[-0.22251689  0.03380726 -0.1890713  -0.33944774 -0.47782856 -0.19175923
    0.35544229  0.14987996 -0.33023643 -0.00491801]]

 [[ 0.0717919   0.          0.13244416 -0.146566   -0.13945419 -0.13636485
   -0.02384129  0.02052689  0.02485587  0.04390082]]

 [[-0.30792588 -0.03342066 -0.3797704  -0.09412873  0.         -0.34681457
   -0.34341204  0.         -0.23057589 -0.14398146]]

 [[-0.40575606 -0.16114083 -0.28730577  0.00579335 -0.1408186  -0.60266638
   -0.61486751  0.          0.         -0.0321773 ]]

 [[-0.24129722  0.10741001  0.02091284 -0.21872473 -0.03098544  0.02319612
   -0.17307937 -0.26030961 -0.31384653 -0.20367689]]

 [[-0.31921017 -0.03592502 -0.20884269  0.         -0.17866668 -0

[[[-0.08941329 -0.09147738 -0.10111408  0.19318604 -0.12476116 -0.14666668
    0.43078727  0.18450114  0.17372479  0.16408493]]

 [[-0.32747528 -0.09171838 -0.63244116 -0.35504529 -0.15369633 -0.70455498
   -0.81146145  0.          0.08192915 -0.10282566]]

 [[-0.80903471 -0.47393954 -0.5114069  -0.33606565 -0.9411366  -0.68997729
   -0.54378104 -0.37358871  0.         -0.16728081]]

 [[-0.20852494 -0.20991182  0.         -0.75740701 -0.45359364 -0.65720224
   -0.10966797 -0.21066654 -0.00523958 -0.17942119]]

 [[ 0.         -0.21464878  0.         -0.24953109 -0.46340978 -0.58292365
   -0.29000902 -0.25766042 -0.18818438 -0.22437626]]

 [[-0.59076834 -0.15254506 -0.29696694 -0.62331021 -0.80892396 -0.21439098
    0.19177277  0.         -0.11400145  0.00441461]]

 [[-0.14120756  0.10288447  0.02301686  0.         -0.64565003 -0.09608394
    0.11475625  0.08324632 -0.10936651  0.07018988]]

 [[-0.007038   -0.01485269  0.05527763  0.         -0.39831105 -0.12364295
    0.         -0.1536

[[[ 0.01636058 -0.13740684  0.08920836 -0.27436259 -0.17764673 -0.26595712
    0.          0.05922446 -0.10704159  0.        ]]

 [[ 0.          0.04701252 -0.32226437 -0.0241041  -0.31767642 -0.05771232
   -0.02055482 -0.06423467 -0.01845828  0.        ]]

 [[ 0.05129015 -0.04268265  0.2358838  -0.02436857  0.14073718  0.06227022
    0.          0.22060136 -0.03284684 -0.08082095]]

 [[-0.31400764 -0.1880772  -0.01952749  0.01960176 -0.47986981 -0.43129826
    0.13580172 -0.2105317  -0.14584261  0.05682665]]

 [[ 0.         -0.09312955 -0.07245961 -0.17194355  0.          0.          0.
   -0.05373533 -0.12648124  0.        ]]

 [[-0.44632426 -0.15305172 -0.11096619 -0.03930739  0.         -0.37299013
   -0.48619017 -0.14908886 -0.13855043  0.        ]]

 [[ 0.          0.06190506 -0.12177133  0.03894514  0.03241743 -0.3247425
   -0.25268266  0.15956236  0.04728153  0.15411887]]

 [[ 0.18276176 -0.13248953  0.08561763  0.25549763 -0.03239706  0.16751829
    0.37970018  0.06163907 -0.0

   -0.02461958  0.          0.00883325 -0.14261848]]]
[[[-0.40161306 -0.21428245  0.02414042 -0.18400002 -0.14976847 -0.2584379
   -0.39717928 -0.06550553  0.00943513 -0.10831205]]

 [[ 0.         -0.1987474   0.         -0.14382762  0.         -0.1799719
   -0.05583639 -0.11878678 -0.04712536 -0.12421355]]

 [[-0.34163925 -0.03728211 -0.01721617 -0.47250694 -0.23733339 -0.53330863
   -0.57066637 -0.08778255 -0.13662454  0.        ]]

 [[-0.29637563 -0.18963319 -0.22084686 -0.27728784  0.         -0.11814718
   -0.15184328 -0.28366363  0.         -0.24252546]]

 [[-0.3289797  -0.07926774 -0.02369793 -0.07333156 -0.58707964  0.
   -0.57066637  0.00587273  0.00244066 -0.2055867 ]]

 [[-0.02998565 -0.09762644  0.         -0.13591293 -0.25092801  0.04700735
    0.07957774  0.13280444  0.06568485 -0.05366022]]

 [[-0.26933336 -0.24588467 -0.00082801 -0.36266667 -0.06883517 -0.1201622
   -0.03308912 -0.32549608 -0.03144168 -0.27183664]]

 [[ 0.          0.         -0.28613442 -0.27704963  0.

[[[ 0.0482748   0.09694641  0.         -0.01837772 -0.0466717  -0.15608878
   -0.02124587  0.05134995  0.06768766  0.11465198]]

 [[-0.06965132 -0.06963459 -0.15471666  0.         -0.15402129 -0.02582861
   -0.31119579 -0.09489619 -0.03221783  0.06435692]]

 [[ 0.1558796   0.11111416 -0.1206062  -0.1506107  -0.18597575  0.27681774
   -0.07937747 -0.15250263  0.         -0.16266388]]

 [[ 0.07359007 -0.038161    0.03082532  0.02188492 -0.1308109   0.13110453
   -0.0208358   0.07892275  0.0701024   0.00443251]]

 [[ 0.          0.         -0.29427773 -0.11016258  0.         -0.08832475
   -0.09411429 -0.34745362 -0.06113156 -0.01416156]]

 [[-0.23686226  0.04980122 -0.20848539 -0.16341099  0.         -0.10593158
   -0.36730966 -0.07334442 -0.01392006 -0.12417128]]

 [[ 0.          0.04631854 -0.06268062  0.00994545 -0.38801783 -0.05003079
   -0.07219385 -0.07517539  0.00590698 -0.11393271]]

 [[ 0.          0.00336756 -0.02399999  0.17701468 -0.09338328 -0.23433462
   -0.08592328 -0.0077